In [3]:
from PIL import Image, ImageEnhance, ImageDraw
import random
import os

def split_image(image, output_size=(224, 224)):
    """
    Dzieli obraz na 4 równe części.
    :param image: Obiekt PIL Image.
    :param output_size: Rozmiar każdego fragmentu po podziale.
    :return: Lista 4 obrazów (PIL Image).
    """
    width, height = image.size
    w_half, h_half = width // 2, height // 2
    return [
        image.crop((0, 0, w_half, h_half)).resize(output_size),  # Lewy górny
        image.crop((w_half, 0, width, h_half)).resize(output_size),  # Prawy górny
        image.crop((0, h_half, w_half, height)).resize(output_size),  # Lewy dolny
        image.crop((w_half, h_half, width, height)).resize(output_size)  # Prawy dolny
    ]

def augment_image(image):
    """
    Wykonuje augmentację obrazu, dodając cienie, zmieniając oświetlenie, itp.
    :param image: Obiekt PIL Image.
    :return: Zaugmentowany obraz (PIL Image).
    """
    # Zmiana jasności
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(random.uniform(0.6, 1.4))

    # Zmiana kontrastu
    #enhancer = ImageEnhance.Contrast(image)
    #image = enhancer.enhance(random.uniform(0.7, 1.3))

    # Odbicie lustrzane
    if random.choice([True, False]):
        image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # Obrót
    angle = random.choice([0, 90, 180, 270])
    image = image.rotate(angle)

    return image

def augment_image_2(image):
    """
    Wykonuje zaawansowaną augmentację obrazu, dodając cienie, brud, zmiany oświetlenia itp.
    :param image: Obiekt PIL Image.
    :return: Zaugmentowany obraz (PIL Image).
    """
    # Zmiana jasności
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(random.uniform(0.6, 1.4))

    # Odbicie lustrzane
    if random.choice([True, False, False, False]):
        image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # Symulacja cienia
    if random.choice([True, False, False, False]):
        image = add_shadow(image)

    # Symulacja brudu
    if random.choice([True, False, False, False]):
        image = add_dirt(image)

    return image

def add_shadow(image):
    """
    Dodaje symulację cienia na losowym fragmencie obrazu.
    :param image: Obiekt PIL Image.
    :return: Obraz z dodanym cieniem.
    """
    draw = ImageDraw.Draw(image)
    width, height = image.size

    # Losowy prostokątny obszar cienia
    x1, y1 = random.randint(0, width // 2), random.randint(0, height // 2)
    x2, y2 = random.randint(width // 2, width), random.randint(height // 2, height)

    # Transparentny czarny cień
    shadow_color = (0, 0, 0, random.randint(50, 100))  # RGB + alpha
    shadow_overlay = Image.new("RGBA", image.size, (0, 0, 0, 0))
    shadow_draw = ImageDraw.Draw(shadow_overlay)
    shadow_draw.rectangle([x1, y1, x2, y2], fill=shadow_color)

    # Nałożenie cienia
    image = Image.alpha_composite(image.convert("RGBA"), shadow_overlay).convert("RGB")
    return image

def add_dirt(image):
    """
    Dodaje symulację brudu (losowych punktów) na obrazie.
    :param image: Obiekt PIL Image.
    :return: Obraz z dodanym brudem.
    """
    draw = ImageDraw.Draw(image)
    width, height = image.size

    # Losowa liczba punktów brudu
    num_dirt = random.randint(5, 15)

    for _ in range(num_dirt):
        x = random.randint(0, width - 1)
        y = random.randint(0, height - 1)

        # Losowy kolor brudu (ciemniejsze odcienie)
        dirt_color = (
            random.randint(50, 100),
            random.randint(50, 100),
            random.randint(50, 100),
        )
        draw.point((x, y), fill=dirt_color)

    return image


def prepare_data(input_dir, output_dir, augmentations_per_image=5):
    """
    Przygotowuje dane przez podział obrazów na 4 fragmenty i wykonanie augmentacji.
    :param input_dir: Ścieżka do folderu z obrazami wejściowymi.
    :param output_dir: Ścieżka do folderu z przetworzonymi danymi.
    :param augmentations_per_image: Liczba augmentacji na każdy fragment.
    """
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.endswith(('.jpg', '.png')):
            image = Image.open(os.path.join(input_dir, filename)).convert('RGB')
            #fragments = split_image(image)
            fragments = [image]

            for i, fragment in enumerate(fragments):
                for j in range(augmentations_per_image):
                    augmented = augment_image_2(fragment)
                    augmented.save(os.path.join(output_dir, f"{filename.split('.')[0]}_{i}_{j}.jpg"))

# Przygotowanie danych
prepare_data(input_dir="../../JETANK/data/mapping/class1", output_dir="../../JETANK/data/mapping/prepared_2/class1", augmentations_per_image=5)
